# 文本数据

In [1]:
import numpy as np
import pandas as pd

## str对象

### str对象的设计意图

`str`对象是定义在`Index`或`Series`上的属性，专门用于**逐元素**处理文本内容。

In [2]:
var = 'abcd'

以下两个命令是等价的：

In [3]:
var.upper()

'ABCD'

In [4]:
str.upper(var)

'ABCD'

对一个`Series`进行文本处理，*首先* 要获取其`str`对象：

In [5]:
s = pd.Series(['abcd','efg','hi'])

In [6]:
# 获取str对象
s.str

pandas的`str`对象方法大部分移植于Python标准库的`str`模块：

In [7]:
# 逐元素进行大写变换
s.str.upper()

0    ABCD
1     EFG
2      HI
dtype: object

### [ ]索引器

通过`[]`索引器，可以取出一个字符串特定位置的元素。

In [8]:
# 颠倒字符串
var[::-1]

'dcba'

In [9]:
# 对字符串构成的Series也是逐元素操作，且超出范围就返回缺失值
s.str[2]

0      c
1      g
2    NaN
dtype: object

### string类型

pandas为序列提供了`string`存储类型，且该类型为`Nullable`类型。

可以通过`astype`方法将一个序列强制转换为`string`类型：

In [10]:
s = pd.Series([12,345,6789])
s

0      12
1     345
2    6789
dtype: int64

In [11]:
# 类型转换
s.astype('string')

0      12
1     345
2    6789
dtype: string

## 正则表达式基础

### 一般字符的匹配

正则表达式是按照某种模式，**从左到右**匹配字符串内容的工具。

In [12]:
import re

In [13]:
# 第一个参数为正则表达式，第二个参数为待匹配字符串
re.findall('Apple','Apple! This Is an Apple!')

['Apple', 'Apple']

### 元字符基础

下表总结了正则表达式中常用的元字符：

|元字符|描述|
|:-----|:---|
|.|匹配除换行符以外的**任意**字符|
|[]|字符类，匹配**方括号中包含的**任意字符|
|[^]|否定字符类，匹配**方括号中不包含**的任意字符|
|* |匹配前面的子表达式**零次或多次**|
|+ |匹配前面的子表达式**一次或多次**|
|? |匹配前面的子表达式**零次或依次**|
|{n,m}|匹配前面字符**至少n次**，但**不超过m次**|
|(xyz)|字符组，按照**确切顺序**匹配字符xyz|
| \| | 分支结构，匹配符号之前的字符**或**后面的字符|
| \\ | 转义符，还原字符原本的含义|
|^ | 匹配**行的开始**|
| \$ |匹配**行的结束**|

### 简写字符集

正则表达式中有一类简写字符集，其等价于**一组字符的集合**：

|简写|描述|
|:---|:---|
|\w |匹配所有**字母、数字、下划线**，即[a-zA-Z0-9_]|
|\W |匹配**非**字母和数字的字符，即[^\w]|
|\d| 匹配**数字**，即[0-9]|
|\D| 匹配**非数字**，即[^\d]|
|\s| 匹配**空格符**，即[\t\n\f\r\p]|
|\S| 匹配**非空格符**，即[^\s]|
|\B| 匹配一组**非空字符 *开头或结尾* 的位置**，不代表具体字符|

## 文本处理的五类操作

### 拆分

`str.split`能够对字符串Seiries进行逐元素拆分，其中第一个参数为**正则表达式**，可选参数为**从左到右**的最大拆分次数`n`，以及是否将拆分结果**展开为多列**`expand`。

In [14]:
s = pd.Series(['上海市黄浦区方浜中路249号','上海市宝山区密山路5号'])

In [15]:
# 将字符串序列拆分，提取市、区、路的信息
s.str.split('[市区]', expand = True)

,0,1,2
0,上海,黄浦,方浜中路249号
1,上海,宝山,密山路5号


### 合并

`str.join`函数能够将字符串列表中的元素连接起来，但如果列表中存在非字符串元素则会返回缺失值：

In [16]:
s = pd.Series([['a','b'], ['c', 'd'], [1, 'a'], [['a', 'b'], 'c']])

In [17]:
s.str.join('-')

0    a-b
1    c-d
2    NaN
3    NaN
dtype: object

`str.cat`函数用于合并两个序列，主要参数为连接符`sep`、连接形式`join`以及缺失值替代符号`na_rep`。其中，连接形式默认为以索引为主键的**左连接**。

In [18]:
s1 = pd.Series(['a','b'])
s2 = pd.Series(['cat','dog'])

In [19]:
s1.str.cat(s2, sep='-')

0    a-cat
1    b-dog
dtype: object

In [20]:
s2.index = [1,2]

In [21]:
s1.str.cat(s2, sep='-', na_rep='?', join='outer')

0      a-?
1    b-cat
2    ?-dog
dtype: object

### 匹配

以下总结了常用的匹配操作：

|命令|效果|是否支持正则表达式|
|:---|:---|:----------------:|
|`str.contains`|判断字符串是否包含指定模式|是|
|`str.startswith`|判断字符串是否以指定模式**开头**|否|
|`str.endswith`|判断字符串是否以指定模式**结束**|否|
|`str.match`|判断字符串是否以指定模式**开头**|是|
|`str.find`|返回**从左到右**第一次匹配的位置索引|否|
|`str.rfind`|返回**从右到左**第一次匹配的位置索引|否|

个人认为最实用的操作是`str.contains`：

In [22]:
s = pd.Series(['my cat', 'he is fat', 'railway station'])
s.str.contains('\s\wat')

0     True
1     True
2    False
dtype: bool

In [23]:
s.str.contains('[f|g]at|n$')

0    False
1     True
2     True
dtype: bool

### 替换

`str.replace`可以实现对字符串序列的逐元素替换。若要使用正则表达式匹配待替换的模式，则应设定参数`regex = True`。

In [24]:
s = pd.Series(['a_1_b','c_?'])
s.str.replace('\d|\?', 'new', regex=True)

0    a_new_b
1      c_new
dtype: object

当字符串元素不同部分的替换要求有差别时，可以使用**子组**的方法，并通过传入**自定义替换函数**来分别进行处理。

In [25]:
s = pd.Series(['上海市黄浦区方浜中路249号',
               '上海市宝山区密山路5号',
               '北京市昌平区北农路2号'])

In [26]:
# 依次匹配市、区、路、号，并将子组命名
pat = '(?P<市名>\w+市)(?P<区名>\w+区)(?P<路名>\w+路)(?P<编号>\d+号)'

In [27]:
# 利用字典定义替换对应关系
city = {'上海市': 'Shanghai', '北京市': 'Beijing'}
district = {'昌平区': 'CP District',
            '黄浦区': 'HP District',
            '宝山区': 'BS District'}
road = {'方浜中路': 'Mid Fangbin Road',
        '密山路': 'Mishan Road',
        '北农路': 'Beinong Road'}

In [28]:
# 定义替换函数
def my_func(m):
    str_city = city[m.group('市名')]
    str_district = district[m.group('区名')]
    str_road = road[m.group('路名')]
    str_no = 'No.' + m.group('编号')[:-1]
    return ' '.join([str_city,str_district,str_road,str_no])

In [29]:
# 完成替换
s.str.replace(pat, my_func, regex = True)

0    Shanghai HP District Mid Fangbin Road No.249
1           Shanghai BS District Mishan Road No.5
2           Beijing CP District Beinong Road No.2
dtype: object

### 提取

`str.extract`可以根据正则表达式指定的模式提取相应的元素。通过子组的命名，可以直接对新生成的DataFrame的列命名。

In [30]:
# 依次提取市、区、路、号
# 定义模式
pat = '(?P<市名>\w+市)(?P<区名>\w+区)(?P<路名>\w+路)(?P<编号>\d+号)'
# 提取
s.str.extract(pat)

,市名,区名,路名,编号
0,上海市,黄浦区,方浜中路,249号
1,上海市,宝山区,密山路,5号
2,北京市,昌平区,北农路,2号


## 常用字符串函数

### 字母型函数

字母型函数主要用于字母的**大小写转换**：

|函数|功能|
|:---|:---|
|str.upper()|将字符串中的**所有**字母转为大写|
|str.lower()|将字符串中的**所有**字母转为小写|
|str.swapcase()|将字符串中的**所有**字母进行大小写互换|
|str.title()|将字符串中的**每一个**单词首字母转为大写，其余部分小写|
|str.capitalize()|将字符串中的**第一个**单词首字母转为大写，其余部分小写|

### 数值型函数

`pd.to_numeric`方法能够将字符格式的数字转换为数值格式，其中`errors`参数定义了对非数值元素的处理模式，有三种选项：`raise`（直接报错）、`coerce`（设为缺失）和`ignore`（保持原状）。

In [31]:
s = pd.Series(['1', '2.2', '2e', '??', '-2.1', '0'])
pd.to_numeric(s, errors='ignore')

0       1
1     2.2
2      2e
3      ??
4    -2.1
5       0
dtype: object

In [32]:
pd.to_numeric(s, errors='coerce')

0    1.0
1    2.2
2    NaN
3    NaN
4   -2.1
5    0.0
dtype: float64

数据清洗时，可以将`errors`参数设为`coerce`，快速查看非数值型的行：

In [33]:
s[pd.to_numeric(s, errors='coerce').isna()]

2    2e
3    ??
dtype: object

### 统计型函数

In [34]:
s = pd.Series(['cat rat fat at', 'get feed sheet heat'])

In [35]:
# 返回特定模式的出现次数
s.str.count('[r|f]at|ee')

0    2
1    2
dtype: int64

In [36]:
# 返回字符串元素的长度
s.str.len()

0    14
1    19
dtype: int64

### 格式型函数

格式型函数主要分为两类，第一种是除空型，第二种时填充型。

除空型函数有三种：`strip`（去除两侧空格）、`rstrip`（去除右侧空格）和`lstrip`（去除左侧空格）。

填充型函数中，`pad`是最灵活的，它可以指定填充后的字符串**长度**、填充的**方向**和填充的**内容**。

In [37]:
s = pd.Series(['a','b','cd','efgh'])
s.str.pad(5,'both','*')

0    **a**
1    **b**
2    **cd*
3    *efgh
dtype: object

In [38]:
s.str.pad(5,'left','*')

0    ****a
1    ****b
2    ***cd
3    *efgh
dtype: object

In [39]:
s.str.pad(5,'right','*')

0    a****
1    b****
2    cd***
3    efgh*
dtype: object

如果要进行**补0**填充，还可以使用`zfill`函数来实现。

In [40]:
s = pd.Series([7, 155, 303000]).astype('string')

In [41]:
s.str.pad(6,'left','0')

0    000007
1    000155
2    303000
dtype: string

In [42]:
# 这种方式更简洁
s.str.zfill(6)

0    000007
1    000155
2    303000
dtype: string

## 练习

### 房屋信息数据集

In [43]:
df = pd.read_excel('data/house_info.xls', usecols=['floor','year','area','price'])

**第1问**

In [44]:
df.year = df.year.str.extract('(\w+)年建')

In [45]:
df.year.head(3)

0    1986
1    2020
2    2010
Name: year, dtype: object

这样的确能够将年份提取出来，但是类型不是整数。如何处理？

In [46]:
df.year[df.year.isna()==False]= df.year[df.year.isna()==False].astype('int')

**第1问参考答案**

参考答案使用了`pd.to_numeric`函数，这种方法很巧妙。后面几问也是类似的方法。另外，对于格式统一且位于开头或结尾的文本提取，不一定使用正则表达式。

``` python
df.year = pd.to_numeric(df.year.str[:-2]).astype('Int64')
```

**第2问**

In [47]:
new = df.floor.str.extract('(?P<Level>\w层)\D+(?P<Highest>\d+)层')

In [48]:
new.Highest[new.Highest.isna()==False]= new.Highest[new.Highest.isna()==False].astype('int')

In [49]:
new = new.convert_dtypes()

In [50]:
df = pd.concat([new,df],axis=1).iloc[:,[0,1,3,4,5]]

In [51]:
df.head()

,Level,Highest,year,area,price
0,高层,6,1986,58.23㎡,155万
1,中层,20,2020,88㎡,155万
2,低层,28,2010,89.33㎡,365万
3,低层,20,2014,82㎡,308万
4,高层,1,2015,98㎡,117万


In [52]:
df = df.convert_dtypes()

In [53]:
df.dtypes

Level      string
Highest     Int64
year        Int64
area       string
price      string
dtype: object

**第3问**

In [54]:
area = df.area.str.extract('(\d+\.*\d*)')

In [55]:
area = area[0].astype('float')

In [56]:
price = df.price.str.extract('(\d+)万')

In [57]:
price = price+'0000'

In [58]:
price = price[0].astype('int')

In [59]:
avg = (price / area).round(0).astype('int').astype('string')

In [60]:
avg = avg+'元/平米'

In [61]:
df['avg_price']=avg

In [62]:
df.head()

,Level,Highest,year,area,price,avg_price
0,高层,6,1986,58.23㎡,155万,26619元/平米
1,中层,20,2020,88㎡,155万,17614元/平米
2,低层,28,2010,89.33㎡,365万,40860元/平米
3,低层,20,2014,82㎡,308万,37561元/平米
4,高层,1,2015,98㎡,117万,11939元/平米


### 《权力的游戏》剧本数据集

In [63]:
df = pd.read_csv('data/script.csv')

In [64]:
df.head(3)

,Release Date,Season,Episode,Episode Title,Name,Sentence
0,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,What do you expect? They're savages. One lot s...
1,2011-04-17,Season 1,Episode 1,Winter is Coming,will,I've never seen wildlings do a thing like this...
2,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,How close did you get?


**第1问**

In [65]:
# 删去列名中的空格
df.columns =  df.columns.str.strip()

In [66]:
# 日期信息多余，shanqu
df = df.iloc[:,1:]

结果如下：

In [67]:
# 分集统计台词
df.groupby(['Season','Episode'])['Sentence'].count()

Season    Episode   
Season 1  Episode 1     327
          Episode 10    266
          Episode 2     283
          Episode 3     353
          Episode 4     404
                       ... 
Season 8  Episode 2     405
          Episode 3     155
          Episode 4      51
          Episode 5     308
          Episode 6     240
Name: Sentence, Length: 73, dtype: int64

**第2问**

In [68]:
# 根据空格拆分台词，并新增一行
df['word_count'] = df.Sentence.str.split(' ').str.len()

In [69]:
word_count = df.groupby('Name')['word_count'].mean()

平均单词量最多的前五个人为：

In [70]:
word_count.sort_values(ascending=False).head(5)

Name
male singer          109.000000
slave owner           77.000000
manderly              62.000000
lollys stokeworth     62.000000
dothraki matron       56.666667
Name: word_count, dtype: float64

**第3问**

大体思路：新增一列，记录上一句台词中的问号数。

In [71]:
# 计算台词问号数，并前移一位
df['nanswer'] = df.Sentence.str.count('\?').shift(1)

In [72]:
df.head(3)

,Season,Episode,Episode Title,Name,Sentence,word_count,nanswer
0,Season 1,Episode 1,Winter is Coming,waymar royce,What do you expect? They're savages. One lot s...,25,NaN
1,Season 1,Episode 1,Winter is Coming,will,I've never seen wildlings do a thing like this...,21,1.0
2,Season 1,Episode 1,Winter is Coming,waymar royce,How close did you get?,5,0.0


回答问题最多的五个人为：

In [73]:
# 分组排序
df.groupby('Name')['nanswer'].sum().sort_values(ascending=False).head(5)

Name
tyrion lannister    527.0
jon snow            374.0
jaime lannister     283.0
arya stark          265.0
cersei lannister    246.0
Name: nanswer, dtype: float64